In [1]:
import open3d as o3d
import numpy as np

# 1. 加载稀疏点云 (假设文件名为 sparse.ply)
pcd = o3d.io.read_point_cloud("D:/docker_test/catkin_ws/src/AirSlam/my_code/new_factory.ply")

# 2. 估计法线（后续的表面重建需要法线信息）
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
pcd.orient_normals_consistent_tangent_plane(30)

# 3. 使用 Poisson 重建生成网格
# depth 参数决定重建的细节层次，根据数据情况调整
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
print("生成的网格顶点数:", np.asarray(mesh.vertices).shape[0])

# 4. 可选：通过密度过滤去除低质量区域
densities = np.asarray(densities)
density_threshold = np.quantile(densities, 0.1)  # 保留密度较高的 90% 顶点
vertices_to_remove = densities < density_threshold
mesh.remove_vertices_by_mask(vertices_to_remove)

# 5. 从网格中采样生成稠密点云（可指定采样点数）
dense_pcd = mesh.sample_points_poisson_disk(number_of_points=100000)
o3d.io.write_point_cloud("dense.ply", dense_pcd)

print("稠密点云已保存为 dense.ply")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
生成的网格顶点数: 98284
稠密点云已保存为 dense.ply
